In [8]:
import torch

from kan import *
from my_functions import calculate_signature, format_signature, calculate_signature_just_autodiff

import numpy as np



In [9]:
# f = lambda x: x[:,[0]]**2 - x[:,[1]]**2
theta = np.pi / 6  # 30 degrees in radians

rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                            [np.sin(theta),  np.cos(theta)]])

f = lambda x: (x @ rotation_matrix)[:,[0]]**2 - (x @ rotation_matrix)[:,[1]]**2

N = 1000
N_sqrt = int(N**0.5)
dataset = create_dataset(f, n_var=2, train_num=N, test_num=N, seed=0)
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [16]:
model = KAN(width=[2,5,1], grid=3, k=5, grid_eps=1.0, seed=0, grid_range=[-1.0,1.0], learn_rotation_mat=True)
model(dataset['train_input'])
# model.plot(beta=100)
model.train(dataset, opt="LBFGS", steps=20, lamb=0.0, lamb_l1=1, lamb_entropy=2.);
# model.plot()
model.prune()
# model.plot(mask=True)
model(dataset['train_input'])
model.train(dataset, opt="LBFGS", steps=50);
mode = "auto" # "manual"

if mode == "manual":
    # manual mode
    model.fix_symbolic(0,0,0,'sin');
    model.fix_symbolic(0,1,0,'x^2');
    model.fix_symbolic(1,0,0,'exp');
elif mode == "auto":
    # automatic mode
    # lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
    lib = ['x','x^2','x^3','x^4']
    model.auto_symbolic(lib=lib)

model.train(dataset, opt="LBFGS", steps=50)
print(model.symbolic_formula()[0][0])


description:   0%|                                                         | 0/20 [7:17:57<?, ?it/s]


KeyboardInterrupt: 

In [11]:
x = torch.tensor([[0.0], [0.0]], requires_grad=True).T
signature = calculate_signature(model, x)
format_signature(signature)


/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  self.acts_scale_std.append(torch.std(postacts, dim=0))


'H: 0.0079, K: -3.9943, H_1: -0.0018, H_2: 0.0051, K_1: 0.0003, K_2: -0.0138'

In [12]:
signature_autodiff = calculate_signature_just_autodiff(model, x)
format_signature(signature_autodiff)

'H: 0.0079, K: -3.9943, H_1: -0.0009, H_2: 0.0053, K_1: 0.0067, K_2: -0.0121'

In [13]:
'H: -0.2460, K: -3.9006, H_1: 1.9603, H_2: -1.4703, K_1: 4.1356, K_2: 3.6560'

'H: -0.2460, K: -3.9006, H_1: 1.9603, H_2: -1.4703, K_1: 4.1356, K_2: 3.6560'

In [15]:
model.alpha

Parameter containing:
tensor([0.2733], requires_grad=True)